In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score

In [2]:
AKL_df = pd.read_csv("./outputs/property_data_with_street.csv",encoding='latin1')
AKL_df = AKL_df.drop(['Unnamed: 0'],axis=1)
print(AKL_df.head())
print(AKL_df.shape)

   CL_QPID  CL_Suburb  CL_Sale_Tenure  CL_Sale_Date  \
0    75499        232               0           229   
1    75639        263               0           119   
2    75639        263               0           169   
3    75639        263               0            70   
4    75639        263               0           162   

   CL_Land_Valuation_Capital_Value  CL_Building_Floor_Area  \
0                        -0.294735               -0.397669   
1                        -0.799770               -0.397669   
2                        -0.799770               -0.397669   
3                        -0.874988               -0.397669   
4                        -0.799770               -0.397669   

   CL_Building_Site_Cover  CL_Land_Area  CL_Bldg_Const  CL_Bldg_Cond  ...  \
0                 0.18143     15.111383             13             1  ...   
1                 0.18143     12.574967              3             1  ...   
2                 0.18143     12.574967              3           

In [3]:
akl_embedding_df = pd.read_csv("./outputs/akl_embedding.csv")
print(akl_embedding_df.shape)
akl_street_nodes_df = pd.read_csv("./outputs/akl_street_nodes.csv")
print(akl_street_nodes_df.shape)
print(akl_street_nodes_df.head(1))

(458252, 64)
(458252, 14)
   street_id     source      target      street_name  street_length  \
0  984794487  279051892  2852049636  South Head Road         32.641   

   restaurant    x    y  Average_POI_Distance  amenity  shop  healthcare  \
0         0.0  0.0  0.0                32.641      0.0   0.0         0.0   

   school  clothes  
0     0.0      0.0  


In [4]:
def find_embedding_for_property(property_df,street_df,emb_df):
    street_emd_columns = ['street_embedding_'+str(i) for i in range(64)]
    output_df = pd.DataFrame([],columns=street_emd_columns)
    count=1
    for row in property_df.iterrows():
        row_index,row_data = row
        street_id,u,v=row_data["street_id"],row_data["street_sources"],row_data["street_targets"]
        street_node = street_df[street_df["street_id"] == street_id]
        street_node = street_node[street_node["source"] == u]
        street_node = street_node[street_node["target"] == v]
        emd_index = street_node.index.values

        current_street_embbeding_dict={}
        embedding_values = emb_df.iloc[emd_index].values[0]
        for i in range(64):
            current_street_embbeding_dict["street_embedding_"+str(i)] = [embedding_values[i]]
        output_df = pd.concat([output_df,pd.DataFrame.from_dict(current_street_embbeding_dict)],ignore_index=True)
        count+=1
        if count % 5000\
                ==0:
            print(count)

    return output_df

In [5]:
embedding_columns = find_embedding_for_property(AKL_df,akl_street_nodes_df,akl_embedding_df)
AKL_df = AKL_df.merge(embedding_columns,left_index=True,right_index=True)
print(AKL_df.head(1))

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000
390000
395000
400000
405000
410000
415000
420000
425000
430000
435000
440000
445000
450000
455000
460000
465000
470000
475000
480000
485000
490000
495000
500000
505000
510000
515000
520000
525000
530000
535000
540000
545000
550000
555000
560000
565000
570000
575000
580000
585000
590000
595000
600000
605000
610000
615000
620000
625000
630000
635000
640000
645000
650000
655000
660000
665000
   CL_QPID  CL_Suburb  CL_Sale_Tenure  CL_Sale_Date  \
0    75499        232             

In [9]:
print(AKL_df.head())
AKL_df.to_csv("./outputs/akl_sale_with_distance_embeddings.csv",index=False)

   CL_QPID  CL_Suburb  CL_Sale_Tenure  CL_Sale_Date  \
0    75499        232               0           229   
1    75639        263               0           119   
2    75639        263               0           169   
3    75639        263               0            70   
4    75639        263               0           162   

   CL_Land_Valuation_Capital_Value  CL_Building_Floor_Area  \
0                        -0.294735               -0.397669   
1                        -0.799770               -0.397669   
2                        -0.799770               -0.397669   
3                        -0.874988               -0.397669   
4                        -0.799770               -0.397669   

   CL_Building_Site_Cover  CL_Land_Area  CL_Bldg_Const  CL_Bldg_Cond  ...  \
0                 0.18143     15.111383             13             1  ...   
1                 0.18143     12.574967              3             1  ...   
2                 0.18143     12.574967              3           

In [16]:
property_columns = ['CL_Suburb','CL_Sale_Tenure','CL_Sale_Date','CL_Land_Valuation_Capital_Value',
                'CL_Building_Floor_Area','CL_Building_Site_Cover',
                'CL_Land_Area','CL_Bldg_Const','CL_Bldg_Cond','CL_Roof_Const','CL_Roof_Cond',
                'CL_Category','CL_LUD_Age','CL_LUD_Land_Use_Description',
                'CL_MAS_No_Main_Roof_Garages','CL_Bedrooms','CL_Bathrooms']+['street_embedding_'+str(i) for i in range(64)]
X_columns = AKL_df[property_columns].values
#print(property_columns)
Y_column = AKL_df['Log_Sale_Price_Net'].values

X_train, X_test, Y_train, Y_test = train_test_split(X_columns, Y_column, test_size = 0.2, random_state = 1,shuffle=True)

print(X_train.shape)

(533812, 81)


In [17]:
hedonic_regression = LinearRegression()
hedonic_regression.fit(X_train, Y_train)

hedonic_regression_training_result = hedonic_regression.predict(X_train)
hedonic_regression_Test_result = hedonic_regression.predict(X_test)

print("\nTraining RMSE:", round(mean_squared_error(Y_train, hedonic_regression_training_result),4))
print("Validation RMSE:", round(mean_squared_error(Y_test, hedonic_regression_Test_result),4))

# (e^0.09) 还原log
print(Y_train,hedonic_regression_training_result)
print(np.exp(Y_train), np.exp(hedonic_regression_training_result))
print("\nTraining RMSE:", round(mean_squared_error(np.exp(Y_train), np.exp(hedonic_regression_training_result)),4))
print("Validation RMSE:", round(mean_squared_error(np.exp(Y_test), np.exp(hedonic_regression_Test_result)),4))

print("\nTraining r2:", round(r2_score(Y_train, hedonic_regression_training_result),4))
print("Validation r2:", round(r2_score(Y_test, hedonic_regression_Test_result),4))

print('----------------------------------------------------------------------')
print('Explained Variance Score of OLS model is {}'.format(explained_variance_score(Y_test,hedonic_regression_Test_result)))


Training RMSE: 0.0899
Validation RMSE: 0.0893
[12.8967167  12.49874226 12.90917016 ... 12.9408415  13.86239414
 12.49125159] [13.06979926 12.0870182  13.08991664 ... 13.00836021 13.30655837
 12.5235701 ]
[ 399000.  268000.  404000. ...  417000. 1048000.  266000.] [474396.74011907 177551.89296013 484037.00517596 ... 446127.56289429
 601125.11659598 274737.14919948]

Training RMSE: 1.8206965939809972e+16
Validation RMSE: 3.4556690275490024e+16

Training r2: 0.845
Validation r2: 0.845
----------------------------------------------------------------------
Explained Variance Score of OLS model is 0.8450219478014234
